# SurrealDBVectorStore

> [SurrealDB](https://surrealdb.com) 是一个统一的多模型数据库，专为人工智能系统而构建。它将结构化和非结构化数据（包括向量搜索、图遍历、关系查询、全文搜索、文档存储和时间序列数据）整合到一个支持 ACID 的单一引擎中，可从 3MB 的边缘二进制文件扩展到云端 PB 级集群。通过消除对多个专用存储的需求，SurrealDB 简化了架构，降低了延迟，并确保了 AI 工作负载的一致性。
>
> **SurrealDB 对 GenAI 系统的意义**
> - **存储和内存的单一引擎：** 在单一系统中结合持久化存储和快速、对代理友好的内存，提供代理所需的所有数据，无需同步多个系统。
> - **代理的单跳内存：** 在单个查询中运行向量搜索、图遍历、语义连接和事务性写入，使 LLM 代理能够快速、一致地访问内存，而无需将关系型、图形和向量数据库拼接在一起。
> - **原地推理和实时更新：** SurrealDB 使代理能够紧邻数据进行推理，并接收毫秒级的新鲜更新，这对于实时推理和协作至关重要。
> - **版本化的持久化上下文：** SurrealDB 支持时间旅行查询和版本化记录，允许代理审计或“重放”过去的 상태 以实现一致、可解释的推理。
> - **即插即用的代理内存：** 将 AI 内存作为原生概念公开，可以轻松地将 SurrealDB 用作 AI 框架的即插即用后端。

本笔记本介绍了如何开始使用 SurrealDB 向量存储。

## 设置

您可以本地运行 SurrealDB，或者先注册一个[免费的 SurrealDB 云账户](https://surrealdb.com/docs/cloud/getting-started)。

本地运行有两种选择：
1. [安装 SurrealDB](https://surrealdb.com/docs/surrealdb/installation) 并[运行 SurrealDB](https://surrealdb.com/docs/surrealdb/installation/running)。通过以下命令运行内存模式：

    ```bash
    surreal start -u root -p root
    ```

2. [通过 Docker 运行](https://surrealdb.com/docs/surrealdb/installation/running/docker)。

    ```bash
    docker run --rm --pull always -p 8000:8000 surrealdb/surrealdb:latest start
    ```

## 安装依赖

安装 `langchain-surrealdb` 和 `surrealdb` Python 包。

```shell
# -- 使用 pip
pip install --upgrade langchain-surrealdb surrealdb
# -- 使用 poetry
poetry add langchain-surrealdb surrealdb
# -- 使用 uv
uv add --upgrade langchain-surrealdb surrealdb
```

要运行此笔记本，我们只需要安装此示例所需的其他依赖项：

In [3]:
!poetry add --quiet --group docs langchain-ollama langchain-surrealdb

## 初始化

In [4]:
from langchain_ollama import OllamaEmbeddings
from langchain_surrealdb.vectorstores import SurrealDBVectorStore
from surrealdb import Surreal

conn = Surreal("ws://localhost:8000/rpc")
conn.signin({"username": "root", "password": "root"})
conn.use("langchain", "demo")
vector_store = SurrealDBVectorStore(OllamaEmbeddings(model="llama3.2"), conn)

## 管理向量存储

### 向向量存储添加项

In [5]:
from langchain_core.documents import Document

_url = "https://surrealdb.com"
d1 = Document(page_content="foo", metadata={"source": _url})
d2 = Document(page_content="SurrealDB", metadata={"source": _url})
d3 = Document(page_content="bar", metadata={"source": _url})
d4 = Document(page_content="this is surreal", metadata={"source": _url})

vector_store.add_documents(documents=[d1, d2, d3, d4], ids=["1", "2", "3", "4"])

['1', '2', '3', '4']

### 更新向量库中的条目

In [6]:
updated_document = Document(
    page_content="zar", metadata={"source": "https://example.com"}
)

vector_store.add_documents(documents=[updated_document], ids=["3"])

['3']

### 从向量存储中删除项目

In [7]:
vector_store.delete(ids=["3"])

## 查询向量存储

一旦你的向量存储创建完成并且已经添加了相关的文档，你很可能希望在运行你的链或代理时查询它。

### 直接查询

执行简单的相似性搜索可以如下进行：

In [8]:
results = vector_store.similarity_search(
    query="surreal", k=1, custom_filter={"source": "https://surrealdb.com"}
)
for doc in results:
    print(f"{doc.page_content} [{doc.metadata}]")  # noqa: T201

this is surreal [{'source': 'https://surrealdb.com'}]


如果你想执行相似性搜索并获得相应的分数，可以运行：

In [9]:
results = vector_store.similarity_search_with_score(
    query="thud", k=1, custom_filter={"source": "https://surrealdb.com"}
)
for doc, score in results:
    print(f"[similarity={score:.0%}] {doc.page_content}")  # noqa: T201

[similarity=57%] this is surreal


### 通过转换为检索器进行查询

您还可以将向量存储转换为检索器，以便在链中使用更加方便。

In [10]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "lambda_mult": 0.5}
)
retriever.invoke("surreal")

[Document(id='4', metadata={'source': 'https://surrealdb.com'}, page_content='this is surreal')]

## 用于检索增强生成的使用方法

有关如何将此向量存储用于检索增强生成 (RAG) 的指南，请参阅以下部分：

- [操作指南：使用 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval/)

## API 参考

关于 SurrealDBVectorStore 所有功能和配置的详细文档，请参阅 API 参考：https://python.langchain.com/api_reference/surrealdb/index.html

## 后续步骤

- 查看 [基础示例](https://github.com/surrealdb/langchain-surrealdb/tree/main/examples/basic)。 使用 Dockerfile 尝试一下！
- 查看 [图示例](https://github.com/surrealdb/langchain-surrealdb/tree/main/examples/graph)
- 尝试 [jupyther notebook](https://github.com/langchain-ai/langchain/blob/master/docs/docs/integrations/vectorstores/surrealdb.ipynb)
- [Awesome SurrealDB](https://github.com/surrealdb/awesome-surreal)，一份精选的 SurrealDB 资源、工具、实用程序和应用程序列表